Tiger/line: census geospatial data
==================================
The  US Census Beureau provides geospatial "shapefiles"
that correspond to the statistical data the collect
and make available through the API.

In this notebook we will demonstrate how to connect to these shapefiles
using `geopandas` and how to **merge** them with the statistical data
to create interactive `folium` maps.


This lab assumes that you have an API key and 
understand the basics of querying the US Census API.


Setup
===============
We will load basic demographic data from the Census API.
See [lab 7](https://drive.google.com/file/d/1SxMqPCqJvNwL4U40bWYWDWF8u5rqAVfF/view?usp=sharing) for more details on how to use the Census API.

The US States
=============



In [1]:
%load_ext autoreload
%autoreload 2


# install libraries that are not part of Colab by default
!pip install census us mapclassify -q



from maptools import tiger, census_vars

# load the libraries we need
from census import Census
import us
import pandas as pd
import geopandas as gpd

# replace this with your own Census API key (this is a fake key)
api_key = "796f9e16b6e3f73329d0d36de60d226d53215cc5"
root = "/home/mxc/Projects/cartopy/tiger"

In [2]:
import os
api_key = os.getenv('CENSUS_API_KEY')


# from google.colab import userdata
# api_key = userdata.get('CENSUS_API_KEY')

api_key
gpd

<module 'geopandas' from '/home/mxc/.virtualenvs/cartopy/lib/python3.10/site-packages/geopandas/__init__.py'>

In [4]:


cv, ct = census_vars._init_vars()
census_vars.search("foo")

dropped foo
a
Number of rows in tables: 27851
Number of rows in tfidf_matrix: 27851
